In [70]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import sys
sys.path.append('/home/justinswinney_snhu/Desktop/Python')
from CrudFunction import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.readMany({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records'))) 
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)


image_filename = 'Grazioso_Salvare_Logo.png' # LOGO
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1(' Grazioso Salvare Dashboard'))),
    html.Div(
        style={'display': 'flex', 'justifyContent': 'center', 'alignItems': 'center'},
        children=[
            html.A(
            href='https://www.snhu.edu/',
            children=[
                html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style={'width': '100px', 'height': 'auto'}
                ),
            ]),
        ]),
    html.Div(
        style={'text-align': 'left', 'bottom':'0', 'left': '0'},
            children=[
                html.P(
                style={'font-size': '12px', 'color': '#808080', 'margin': '0'},
                    children=[
                        'By: Justin Swinney'],
                ),
            ]),

        
    html.Hr(),
    html.Div(
        style={'display': 'flex', 'justifyContent': 'space-between','alignItems':'flex-end'},
        children=[
            dcc.RadioItems(
                id='filter-type',
                options=[
                    {'label': 'Water Rescue', 'value': 'water_rescue'},
                    {'label': 'Wilderness Rescue', 'value': 'wilderness_rescue'},
                    {'label': 'Disaster Tracking', 'value': 'disaster_tracking'},
                    {'label': 'Reset', 'value': 'reset'}
                ],
                     inline=True,
            labelStyle={'margin-right': '20px'}),
            dcc.Input(
            id='search-bar',
            type='text',
            placeholder='search breed..',
            style={'margin-right': '10px'})
        ]),
        
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         page_size = 10,
                         style_table = {'overflowX': 'auto'},
                         row_selectable = 'single',
                         selected_rows = [0],
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value'),
              Input('search-bar', 'value')])

### UPDATE RADIO FILTERS AND SEARCH BAR ###

def update_dashboard(filter_type, search_query):
    try:
        df = pd.DataFrame.from_records(db.readMany({}))
        update_graphs(df)
        if filter_type == 'reset':
            df = pd.DataFrame.from_records(db.readMany({})) #Read all unfiltered 
            update_graphs(df)# updating graph
        elif filter_type == 'water_rescue':
            water_breeds = ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]
            query = createQuery("Dog", water_breeds, "Intact Female", 26, 156)
            df = pd.DataFrame.from_records(db.readMany(query)) #Read all from Water Rescue
            update_graphs(df)# updating graph
        elif filter_type == 'wilderness_rescue':
            wilderness_breeds = ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]
            query = createQuery("Dog", wilderness_breeds, "Intact Male", 26, 156)
            df = pd.DataFrame.from_records(db.readMany(query)) #Read all from Wilderness Rescue
            update_graphs(df)# updating graph
        elif filter_type == "disaster_tracking":
            disaster_breeds = ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]
            query = createQuery("Dog", disaster_breeds, "Intact Male", 20, 300)
            df = pd.DataFrame.from_records(db.readMany(query)) #Read all from Disaster Rescue
            update_graphs(df)# updating graph

        if search_query:
            df = df[df['breed'].str.contains(search_query, case=False, na=False)] #searching in column breed 
            update_graphs(df) # updating graph

        df.drop(columns=['_id'], inplace=True) # Drop columns to only retrieve desired list 
        return df.to_dict('records')
    except Exception as e:
        return []

    
## Function for creating querys.
def createQuery(animal_type, breeds, sex, min_age, max_age): # takes in type,breeds, sex,min and max age.
    query = {"animal_type": animal_type,"breed":{"$in": breeds}, "sex_upon_outcome": sex, "age_upon_outcome_in_weeks": {"$gte": min_age, "$lte": max_age}}
    return query

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return []
    return [
        dcc.Graph(            
            figure = px.pie(viewData, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:19796/
